In [48]:
import numpy as np
import cvxpy as cp
from scipy.optimize import linprog
from sklearn.linear_model import QuantileRegressor

In [49]:
X = np.c_[np.ones(100), np.random.normal(0, 5, size=(100, 5))]
beta = np.array([10, 1, 2, 3, 4, 5])
y = X @ beta + np.random.normal(0, 1, 100)

In [50]:
# print (cp.installed_solvers())

In [58]:
class QuantileRegression:
    """
    Original form:
        
        min. Σ max[(q-1) * (y - yhat)_i, q * (y - yhat)_i]
    
    where y_hat can be estimated by 4 CAViaR functions:
    1. Adaptive:     f(B1)_t = f(B1)_t-1 + B1 * {[1 + exp(G * [y_t-1 - f(B1)_t-1])]^-1 - q}
    2. Symmetric:    f(B)_t = B1 + B2 f(B)_t-1 + B3 |y_t-1|
    3. Assymmetric:  f(B)_t = B1 + B2 f(B)_t-1 + B3 max(y_t-1, 0) + B4 min(y_t-1, 0)
    4. IGRACH(1, 1): f(B)_t = (B1 + B2 f(B)_t-1**2 + B3 y_t-1**2)**0.5
    """
    def __init__(self, theta, tol=1e-10):
        self.theta = theta # theta-quantile
        self.tol = tol # tolerance for loss to break the loop
    
    def fit(self, X, y, fit_intercept=False):
        """
        if X == None => autoregressive; else normal regerssion
        """
        # data preparation
        self.n, self.p = X.shape
        
        # if data itself doesnt contain intercept term
        if fit_intercept:
            self.p += 1
            X = np.c_[np.ones(self.n), X]
            
        # initialize the params to recursively fit the data
        loss = np.inf
        beta_hat = cp.Variable(self.p)
        
        loss_fct = cp.sum(cp.maximum(self.theta * (y - X @ beta_hat),
                                     (self.theta - 1) * (y - X @ beta_hat)))
        prob = cp.Problem(cp.Minimize(loss_fct))
        prob.solve(solver='ECOS')
        
        self.params = beta_hat.value

    def autoregressive_fit(self, returns):
        self.n = returns.shape[0]
        first = True
        count = 0
        
        params = np.random.uniform(0, 1, 4)
        sigmas = abs(np.random.normal(0, 1, self.n+1))
        
        constant_term = np.ones_like(returns)
        while True:
            if not first:
                sigmas = self.asymmetric_slope(returns, self.params)
            else:
                first = False
            X = np.c_[constant_term, sigmas[:-1], np.maximum(returns, 0), np.minimum(returns, 0)]
            y = sigmas[1:]
            self.fit(X, y)
            print(self.params)
            
            if count == 2000:
                break
            count += 1

    def fit_LP(self, X, y, fit_intercept=True):
        """
        Epigraph form (LP):

            min. 1Tt
            s.t. max[(q-1) * (y - yhat), q * (y - yhat)]_i <= ti
        
        let mxi == max[(q-1) * (y - yhat), q * (y - yhat)]_i
        since mxi is always a positive value, it implies mxi <= ti only
        
        if y - y_hat positive => q; else q - 1
        """       
        # data preparation
        self.n, self.p = X.shape
        
        # if data itself doesnt contain intercept term
        if fit_intercept:
            self.p += 1
            X = np.c_[np.ones(self.n), X]
            
        # initialize the params to recursively fit the data
        self.params = np.random.uniform(0, 1, self.p)
        
        # fit the data
        loss = np.inf
        
        for i in range(10000):
            # calculate the deviations
            y_hat = X @ self.params
            dev = y - y_hat
            
            # the lower bounds of t
            lowers = np.maximum((self.theta-1) * (dev), self.theta * (dev))
            bounds = [(lb, None) for lb in lowers]
            
            # solve the above LP
            res = linprog(np.ones_like(y), bounds=bounds)
            if loss - res.fun <= self.tol:
                break
            loss = res.fun
            
            # update the betas
            t = res.x
            print(t)
            q_indicator = np.where(dev>0, self.theta, self.theta-1)
            self.params = np.linalg.pinv(X) @ (y - t / q_indicator)
        
    def loss(self, y, y_hat):
        # sum of absolute error
        dev = y - y_hat
        return np.sum(np.maximum(self.theta * dev, (self.theta - 1) * dev))

    def asymmetric_slope(self, returns, params):
        b1, b2, b3, b4 = params
        sigmas = np.zeros(returns.shape[0]+1)
        sigmas[0] = np.std(returns) # im not sure
        for t in range(1, len(sigmas)):
            sigmas[t] = b1 + b2 * sigmas[t-1] + max(b3 * returns[t-1], 0) + b4 * min(b3 * returns[t-1], 0)
        return sigmas

In [59]:
X = np.c_[np.ones(100), np.random.normal(0, 5, size=(100, 5))]
beta = np.array([100, -1, 2, -3, 4, -5])
y = X @ beta + np.random.normal(0, 1, 100)

In [60]:
qr = QuantileRegression(0.01)
qr_sklearn = QuantileRegressor(quantile=0.01, alpha=0, fit_intercept=False)

In [61]:
 np.linalg.pinv(X).shape

(6, 100)

In [62]:
qr.fit_LP(X, y)

[1.46134787 1.05420943 0.808084   0.937492   1.25928168 1.20604219
 0.45569092 1.41677203 1.3157444  0.50760378 0.43726618 0.68653582
 0.8280005  1.05974604 0.74761387 1.22899198 0.93627466 1.88326718
 0.70227945 0.1348207  1.00116668 0.45385496 1.2591387  0.76296697
 0.86475829 1.322727   0.79923992 1.18585306 0.58977195 0.74853575
 1.06948937 1.0866337  1.67637483 1.34327459 1.81636543 1.01483955
 0.44262206 1.1229811  1.79615333 0.69470111 1.03034534 0.27306248
 0.96452925 1.36796246 1.33055761 1.75536394 0.58644957 1.4100492
 1.66225668 1.0785576  0.2351878  0.82313353 0.71814478 0.83847303
 1.19592557 1.42783255 1.8745864  1.18054043 0.92346383 0.67458904
 0.7153207  0.92048951 1.13837132 0.977481   1.13283254 1.4775084
 1.42908846 1.02274802 1.06024246 0.49431041 1.54792029 1.32335962
 0.55199818 0.66808087 1.30826464 1.33124639 1.48415889 0.41703558
 0.92144626 1.16072397 1.0993613  1.32606427 0.32366245 0.91024249
 0.74789447 1.57195799 0.64073398 0.48622944 1.25111542 1.320904

In [47]:
qr.autoregressive_fit(y)

[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]
[-0.25842028  0.15637977  0.00141575  0.        ]


[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]
[-0.2584203   0.15637977  0.00141575  0.        ]


[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]
[-0.25842032  0.15637977  0.00141575  0.        ]


[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]
[-0.25842034  0.15637977  0.00141575  0.        ]


[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]
[-0.25842036  0.15637977  0.00141575  0.        ]


...

[-0.25842041  0.15637977  0.00141575  0.        ]
[-0.25842041  0.15637977  0.00141575  0.        ]
[-0.25842041  0.15637977  0.00141575  0.        ]
[-0.25842041  0.15637977  0.00141575  0.        ]
[-0.25842041  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]
[-0.25842042  0.15637977  0.00141575  0.        ]


[-0.25842043  0.15637977  0.00141575  0.        ]
[-0.25842043  0.15637977  0.00141575  0.        ]
[-0.25842043  0.15637977  0.00141575  0.        ]
[-0.25842043  0.15637977  0.00141575  0.        ]
[-0.25842043  0.15637977  0.00141575  0.        ]
[-0.25842043  0.15637977  0.00141575  0.        ]
[-0.25842043  0.15637977  0.00141575  0.        ]
[-0.25842043  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]
[-0.25842044  0.15637977  0.00141575  0.        ]


[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842045  0.15637977  0.00141575  0.        ]
[-0.25842046  0.15637977  0.00141575  0.        ]
[-0.25842046  0.15637977  0.00141575  0.        ]
[-0.25842046  0.15637977  0.00141575  0.        ]
[-0.25842046  0.15637977  0.00141575  0.        ]
[-0.25842046  0.15637977  0.00141575  0.        ]
[-0.25842046  0.15637977  0.00141575  0.        ]
[-0.25842046  0.15637977  0.00141575  0.        ]
[-0.25842046  0.15637977  0.00141575  0.        ]


[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842047  0.15637977  0.00141575  0.        ]
[-0.25842048  0.15637977  0.00141575  0.        ]
[-0.25842048  0.15637977  0.00141575  0.        ]
[-0.25842048  0.15637977  0.00141575  0.        ]
[-0.25842048  0.15637977  0.00141575  0.        ]
[-0.25842048  0.15637977  0.00141575  0.        ]
[-0.25842048  0.15637977  0.00141575  0.        ]
[-0.25842048  0.15637977  0.00141575  0.        ]
[-0.25842048  0.15637977  0.00141575  0.        ]


[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.25842049  0.15637977  0.00141575  0.        ]
[-0.2584205   0.15637977  0.00141575  0.        ]
[-0.2584205   0.15637977  0.00141575  0.        ]
[-0.2584205   0.15637977  0.00141575  0.        ]
[-0.2584205   0.15637977  0.00141575  0.        ]
[-0.2584205   0.15637977  0.00141575  0.        ]


In [34]:
qr.fit(X, y, fit_intercept=False)

In [ ]:
qr_sklearn.fit(X, y)

In [29]:
qr.params

array([98.31215737, -0.94949759,  1.9675716 , -2.94511591,  4.06207583,
       -4.94583376])

In [30]:
qr_sklearn.coef_

array([98.31215737, -0.94949759,  1.9675716 , -2.94511591,  4.06207583,
       -4.94583376])